In [64]:
import pydicom

import matplotlib.pyplot as plt
import SimpleITK as sitk
import pydicom
import numpy as np
import os
from glob import glob

from pydicom.dataset import FileMetaDataset

from pydicom.uid import UID


import numpy as np
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ImplicitVRLittleEndian, ExplicitVRLittleEndian
import time

In [65]:
def nii_to_single_dicom(nii_path_list, dicom_output_path, template_dicom_path="heart-dicom-rest-gate.dcm"):
    
    image_array = np.zeros((648, 80, 80), dtype=np.float32)
    for i, nii_path in enumerate(nii_path_list):
        # 读取 NIfTI 文件
        image = sitk.ReadImage(nii_path)
        image_array[i*81: (i+1)*81] = sitk.GetArrayFromImage(image)  # (Z, Y, X)
    
    image_array = image_array.astype(np.uint16)

    # 创建文件元数据
    file_meta = pydicom.dataset.FileMetaDataset()
    file_meta.TransferSyntaxUID = ExplicitVRLittleEndian  # 设置传输语法为 Explicit VR Little Endian

    # 创建 FileDataset 对象
    ds = FileDataset(dicom_output_path, {}, file_meta=file_meta, preamble=b"\0" * 128)

    # 设置一些基本 DICOM 标签（可以根据需求调整）
    ds.PatientName = "Test^Patient"
    ds.PatientID = "123456"
    ds.StudyInstanceUID = "1.2.3.4"
    ds.SeriesInstanceUID = "1.2.3.5"
    ds.SOPInstanceUID = "1.2.3.6"
    ds.Modality = "SPECT"
    ds.Manufacturer = "My Manufacturer"

    # 设置图像数据（存储为原始的 float32）
    ds.PixelData = image_array.tobytes()

    # 设定图像的元数据
    ds.NumberOfSlices, ds.Rows, ds.Columns = image_array.shape  # 更新为 3D 图像的行数、列数和切片数
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"  # 或者 "RGB" 依据图像类型

    # # 添加缺失的 DICOM 标签
    # ds.BitsAllocated = 32  # 设置为 float32 类型
    # ds.BitsStored = 32    # 存储 32 位
    # ds.HighBit = 31       # 高位位置
    # ds.PixelSpacing = [1.0, 1.0]  # 仅作示例，依据需求设置
    # ds.PixelRepresentation = 0  # 对于 float32，它是浮点类型，通常无符号
    
    # 添加缺失的 DICOM 标签
    ds.BitsAllocated = 16  # 设置为 uint16 类型
    ds.BitsStored = 16    # 存储 16 位
    ds.HighBit = 15       # 高位位置
    ds.PixelSpacing = [1.0, 1.0, 1.0]  # 仅作示例，依据需求设置
    ds.PixelRepresentation = 0  # 对于 uint16，它是无符号整数类型

    # 保存为 DICOM 文件
    ds.save_as(dicom_output_path)
    print(f"转换完成, DICOM 文件已保存到 {dicom_output_path}")


base_dir = "/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/gate_recon"
dst_dir = "/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/gate_recon_dicom"


for i in range(10):
    nii_path_list = glob(os.path.join(base_dir, f"{i}_*_i5.nii.gz"))
    
    nii_path_list = sorted(nii_path_list)
    out_filepath = os.path.join(dst_dir, f"{i}_gate.dcm")
    # out_filepath = os.path.join(f"{i}_gate.dcm")
    nii_to_single_dicom(nii_path_list, out_filepath)
    break

转换完成, DICOM 文件已保存到 /mnt/labnas/DataCollection/Project/RadiSmart/cardiac/gate_recon_dicom/0_gate.dcm


In [69]:

dicom_path = "heart-dicom-rest-gate.dcm"
ds1 = pydicom.dcmread(dicom_path)

In [72]:
ds1.pixel_array.shape, ds1.pixel_array.dtype

((648, 80, 80), dtype('float32'))

In [ ]:
arr = np.zeros((648, 80, 80), dtype=np.float32)

# ds1.
del ds1[0x7FE0,0x0008]
# print([0x7FE0,0x0008] in ds1)
pydicom.pixels.set_pixel_data(ds1, arr, "MONOCHROME2", 32)

ds1.remove_private_tags

False


ValueError: Unsupported ndarray dtype 'float32', must be int8, int16, uint8 or uint16

In [68]:
# filename = "out32.dcm"

dicom_path = "heart-dicom-rest-gate.dcm"
ds1 = pydicom.dcmread(dicom_path)

ds = FileDataset(filename, {}, file_meta=ds1.file_meta, preamble=b"\0" * 128)

nii_path_list = glob(os.path.join(base_dir, f"{i}_*_i5.nii.gz"))
nii_path_list = sorted(nii_path_list)

image_array = np.zeros((648, 80, 80), dtype=np.float32)
for i, nii_path in enumerate(nii_path_list):
    # 读取 NIfTI 文件
    image = sitk.ReadImage(nii_path)
    image_array[i*81: (i+1)*81] = sitk.GetArrayFromImage(image)  # (Z, Y, X)



In [60]:
dicom_path = "heart-dicom-rest-gate.dcm"
ds1 = pydicom.dcmread(dicom_path)

ds1.file_meta


(0002,0000) File Meta Information Group Length  UL: 178
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: Secondary Capture Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 2.25.199861141585367010503236253734849631363
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002,0013) Implementation Version Name         SH: 'fo-dicom 5.1.3'

In [62]:
# 读取 DICOM 文件
dicom_path = "/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/gate_recon_dicom/0_gate.dcm"
ds2 = pydicom.dcmread(dicom_path)

ds2.file_meta

(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian